# Upload Feature Importance Values to Fiddler Model

This notebook demonstrates how to upload user-defined feature importance (impact) scores to a Fiddler model using the `fiddler_utils` package.

## Use Case

When you have pre-computed global feature importance values (e.g., from SHAP analysis, permutation importance, or other methods), you can upload them to Fiddler to populate the feature impact charts without needing to provide a model artifact.

## Prerequisites

* Fiddler instance access with API token
* Model already registered in Fiddler
* CSV file with feature importance values (feature_name, feature_impact)

## Resources

* [API Documentation](https://docs.fiddler.ai/technical-reference/api-methods-30#upload_feature_impact)
* [Quickstart Example](https://github.com/fiddler-labs/fiddler-examples/blob/main/quickstart/latest/Fiddler_Quickstart_User_Defined_Feature_Impact.ipynb)

In [ ]:
%pip install -q fiddler-client

import sys
sys.path.insert(0, '..')  # Add parent directory to path for fiddler_utils

import fiddler as fdl
from fiddler_utils import get_or_init, FeatureImpactManager

print(f'Fiddler client version: {fdl.__version__}')

## Configuration

In [ ]:
# Fiddler connection details
URL = 'https://customer.fiddler.ai'
AUTH_TOKEN = ''  # Get this from the Settings > Credentials tab in Fiddler UI

# Model identification - use either MODEL_ID or PROJECT_NAME + MODEL_NAME
MODEL_ID = ''  # Option 1: Get model by ID

# Option 2: Get model by project and name
PROJECT_NAME = ''
MODEL_NAME = ''
VERSION = ''  # Optional, leave empty for unversioned models

# Feature importance data source
FEATURE_IMPORTANCE_JSON = '../quickstart/data/custom_feature_impact_scores_alt.json'
FEATURE_IMPORTANCE_JSON_ALT = '../quickstart/data/custom_feature_impact_scores.json'

# Option 3: Define directly as dictionary
# FEATURE_IMPORTANCE_DICT = {
#     'age': 0.25,
#     'income': 0.32,
#     ...
# }


## Initialize Connection and Get Model

In [ ]:
# Initialize connection with error-level logging
get_or_init(url=URL, token=AUTH_TOKEN, log_level="ERROR")

print(f"Successfully connected to Fiddler at {URL}")

In [ ]:
# Get model using either MODEL_ID or PROJECT_NAME + MODEL_NAME
if MODEL_ID:
    model = fdl.Model.get(id_=MODEL_ID)
    print(f"Retrieved model by ID: {model.name}")
else:
    project = fdl.Project.from_name(name=PROJECT_NAME)
    model_kwargs = {
        'name': MODEL_NAME,
        'project_id': project.id
    }
    if VERSION:
        model_kwargs['version'] = VERSION
    
    model = fdl.Model.from_name(**model_kwargs)
    print(f"Retrieved model: {model.name} (version: {model.version or 'none'})")

print(f"Model ID: {model.id}")
print(f"Number of input features: {len(model.spec.inputs)}")

## Load Feature Importance Values

The `FeatureImpactManager` supports loading from:
* CSV files (with configurable column names)
* JSON files
* Python dictionaries

In [ ]:
# Initialize the FeatureImpactManager
fi_mgr = FeatureImpactManager()

# Option 1: Load from JSON
feature_impact_dict = fi_mgr.load_from_json(
    filepath=FEATURE_IMPORTANCE_JSON
)

print(f"\nLoaded {len(feature_impact_dict)} feature impact scores")
print(f"\nSample of loaded values:")
for feature, impact in list(feature_impact_dict.items())[:5]:
    print(f"  {feature}: {impact}")

## Validate Feature Impact Values

Validation ensures:
* All model input features have impact scores (REQUIRED)
* Extra features (not in model spec) are identified and can be removed
* All values are numeric

In [ ]:
# Validate features against model spec
print("Validating feature impact values...\n")

is_valid, missing_features, extra_features = fi_mgr.validate_features(
    model=model,
    feature_impact_dict=feature_impact_dict,
    remove_extra=True,  # Automatically remove features not in model spec
    strict=False  # Don't raise exception yet, just report
)

# Report validation results
print(f"Model has {len(model.spec.inputs)} input features")
print(f"Impact map has {len(feature_impact_dict)} features")
print()

if missing_features:
    print(f"⚠️  Missing features (in model but not in impact map):")
    for feature in sorted(missing_features):
        print(f"   - {feature}")
    print()
else:
    print("✅ All model input features have impact scores")
    print()

if extra_features:
    print(f"ℹ️  Extra features (in impact map but not in model):")
    for feature in sorted(extra_features):
        print(f"   - {feature}")
    print(f"   These have been removed from the impact map.")
    print()

if is_valid:
    print("✅ Validation passed! Ready to upload.")
else:
    print("❌ Validation failed. Please add impact scores for missing features.")

## Upload Feature Impact Scores to Fiddler

Upload the validated feature impact scores to the model.
* Set `update=True` to update existing values
* Set `update=False` to fail if values already exist

In [ ]:
# Upload feature impact scores (auto-detect mode)
# The manager will try creating first, then update if values already exist
print("Uploading feature impact scores to Fiddler...\n")

try:
    result = fi_mgr.upload_feature_impact(
        model=model,
        feature_impact_map=feature_impact_dict,
        # update=None (default) auto-detects: tries create first, updates if exists
        validate=True,
        remove_extra=True
    )
    
    print(f"✅ Successfully uploaded {len(result)} feature impact values to model '{model.name}'")
    print(f"\nFeature impact scores are now available in the Fiddler UI.")
    print(f"Navigate to: {URL}/projects/{model.project_id}/models/{model.id}")
    
except Exception as e:
    print(f"❌ Upload failed: {str(e)}")

## Update Feature Impact Values (Optional)

This cell demonstrates updating existing feature impact values with new scores from an alternative file.
The `upload_feature_impact()` method will automatically detect that values exist and update them.

In [ ]:
# Load and upload alternative feature impact scores
print("Loading alternative feature impact scores...\n")

updated_impact_dict = fi_mgr.load_from_json(filepath=FEATURE_IMPORTANCE_JSON_ALT)
print(f"Loaded {len(updated_impact_dict)} alternative scores")

# Show comparison before uploading
print(f"\nComparison (first 5 features):")
for feature in list(updated_impact_dict.keys())[:5]:
    old_val = feature_impact_dict.get(feature, 'N/A')
    new_val = updated_impact_dict.get(feature, 'N/A')
    print(f"  {feature}: {old_val} → {new_val}")

# Upload with auto-detect (will update existing values)
print(f"\nUploading alternative scores...")
try:
    result = fi_mgr.upload_feature_impact(
        model=model,
        feature_impact_map=updated_impact_dict,
        validate=True,
        remove_extra=True
    )
    print(f"✅ Successfully uploaded {len(result)} updated feature impact values")
    print(f"\nThe feature impact charts in Fiddler UI now reflect the new scores.")
    
except Exception as e:
    print(f"❌ Upload failed: {str(e)}")

## Summary

This notebook demonstrated:
* Loading feature importance values from CSV using `FeatureImpactManager`
* Validating features against model spec
* Handling missing and extra features
* Uploading feature impact scores to Fiddler

### Next Steps

* View feature impact charts in the Fiddler UI
* Update values by running this notebook again with `update=True`
* Try different feature importance calculation methods (SHAP, permutation, etc.)

### Resources

* [User-Defined Feature Impact Quickstart](https://github.com/fiddler-labs/fiddler-examples/blob/main/quickstart/latest/Fiddler_Quickstart_User_Defined_Feature_Impact.ipynb)
* [Fiddler Documentation](https://docs.fiddler.ai/)
* [upload_feature_impact API Reference](https://docs.fiddler.ai/technical-reference/api-methods-30#upload_feature_impact)